In [1]:
import csv
import pandas as pd
import re

In [2]:
df = pd.read_csv('NB_predict_oily.csv')
df.head()

,Unnamed: 0,aspect,sentence,review,rating,product_name,word_only,predict
0,0,트러블 진정,"[('총', 'MM'), ('평', 'NNB'), ('트러블', 'NNP'), ('...",닥터지 레드블레미쉬 클리어 수딩크림을 벌써 몇 통째 사용중인지 모르겠어요 피부가 민...,5,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"['총', '평', '트러블', '피부', '나', '민감', '피부', '도', ...",1
1,1,트러블 진정,"[('주', 'VX'), ('는', 'ETM'), ('느낌', 'NNG'), ('이...",닥터지 레드블레미쉬 클리어 수딩크림을 벌써 몇 통째 사용중인지 모르겠어요 피부가 민...,5,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"['주', '는', '느낌', '이', '라', '지성', '민감성', '트러블',...",1
2,2,수분/보습,"[('수분', 'NNP'), ('이', 'JKS'), ('많', 'VA'), ('아...",성분좋은 수분크림 찾다가 구매한 제품인데 정말 만족하고있습니다 수분이 많아서 촉촉하...,5,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"['수분', '이', '많', '아서']",0
3,3,트러블 진정,"[('트러블', 'NNP'), ('피부', 'NNG'), ('에', 'JKB'), ...",성분좋은 수분크림 찾다가 구매한 제품인데 정말 만족하고있습니다 수분이 많아서 촉촉하...,5,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"['트러블', '피부', '에', '좋', '은', '것', '같', '습니다']",0
4,4,수분/보습,"[('피부타입', 'NNP'), ('세', 'MM'), ('지성', 'NNP'), ...",피부타입 세 지성 좁쌀 성인 여드름 얼굴 외곽선 쪽에만 수분 부족 민감성 입니다 스...,4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"['피부타입', '세', '지성', '좁쌀', '성인', '여드름', '얼굴', '...",0


### 지성 전체분석

In [3]:
df['predict'].value_counts()

1    1658
0    1093
Name: predict, dtype: int64

In [4]:
df['aspect'].value_counts()

트러블 진정    1042
수분/보습      924
가성비        506
피부개선       279
Name: aspect, dtype: int64

In [5]:
product_list=[]

In [6]:
product_list = df['product_name'].unique()

In [7]:
len(product_list)

223

In [8]:
df.loc[df['aspect'] == '트러블 진정']['predict'].value_counts()

1    546
0    496
Name: predict, dtype: int64

In [9]:
df.loc[df['aspect'] == '수분/보습']['predict'].value_counts()

1    587
0    337
Name: predict, dtype: int64

In [10]:
df.loc[df['aspect'] == '가성비']['predict'].value_counts()

1    352
0    154
Name: predict, dtype: int64

In [11]:
df.loc[df['aspect'] == '피부개선']['predict'].value_counts()

1    173
0    106
Name: predict, dtype: int64

----------------------

### 제품별 분석

In [12]:
senti_product = pd.DataFrame(columns=['product_name', 'sooth','moisture','cost','func'])

In [13]:
for p in product_list:
    df_= df.loc[df['product_name']== str(p)]
    t_li = [0,0]
    mo_li = [0,0]
    co_li = [0,0]
    f_li = [0,0]
    
    for i in df_.loc[df_['aspect'] == '트러블 진정']['predict']:
        t_li[i] += 1
    
    for i in df_.loc[df_['aspect'] == '수분/보습']['predict']:
        mo_li[i] += 1
    
    for i in df_.loc[df_['aspect'] == '가성비']['predict']:
        co_li[i] += 1
        
    for i in df_.loc[df_['aspect'] == '피부개선']['predict']:
        f_li[i] += 1
        
    new_data = {
        'product_name': p,
        'sooth': t_li,
        'moisture': mo_li,
        'cost': co_li,
        'func': f_li
    }
    
    senti_product = senti_product.append(new_data, ignore_index=True)


In [14]:
senti_product

,product_name,sooth,moisture,cost,func
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),"[15, 19]","[9, 16]","[9, 8]","[0, 0]"
1,에스트라 아토베리어365 크림 80ml 짝꿍기획 (+하이드로에센스 40ml 증정),"[3, 3]","[6, 7]","[0, 0]","[0, 0]"
2,[단독][1+1] 크리니크 쏙보습크림 50ml 1+1 기획세트 (+애슝 스티커 증정),"[0, 0]","[0, 5]","[0, 0]","[0, 0]"
3,"라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)","[15, 18]","[0, 1]","[1, 6]","[0, 0]"
4,[오원Pick] 닥터자르트 시카페어 크림(50ml+25ml) 한정세트,"[5, 4]","[2, 2]","[2, 8]","[0, 0]"
...,...,...,...,...,...
218,메이크프렘 세이프 미 릴리프 인텐시브 CP 크림 80ml,"[3, 1]","[3, 3]","[2, 1]","[0, 0]"
219,피지오겔 레드수딩 AI 인텐시브 리피드크림 50ml,"[0, 2]","[0, 0]","[1, 0]","[0, 0]"
220,웰라쥬 리얼 히알루로닉 겔크림 75ml,"[0, 0]","[0, 1]","[0, 0]","[0, 0]"
221,닥터디퍼런트 131 모이스처라이저 100ml,"[0, 0]","[0, 0]","[0, 0]","[0, 1]"


In [28]:
senti_product.to_csv('senti_product_oily.csv', index = False, columns = ['product_name', 'sooth','moisture','cost','func'])

---------------------

### 속성별 언급 비율 (%)

In [16]:
senti_aspect_percent = pd.DataFrame(columns=['product_name', 'sooth','moisture','cost','func'])

In [18]:
for i in range(len(senti_product)):
    s_per=0
    m_per=0
    c_per=0
    f_per=0

    total = sum(senti_product['sooth'][i])+sum(senti_product['moisture'][i])+sum(senti_product['cost'][i])+sum(senti_product['func'][i])
    
    s_per=round(sum(senti_product['sooth'][i])/total, 4)
    m_per=round(sum(senti_product['moisture'][i])/total,4)
    c_per=round(sum(senti_product['cost'][i])/total,4)
    f_per=round(sum(senti_product['func'][i])/total,4)
    
    new_data = {
        'product_name': senti_product['product_name'][i],
        'sooth': s_per*100,
        'moisture': m_per*100,
        'cost': c_per*100,
        'func': f_per*100
    }
    
    senti_aspect_percent = senti_aspect_percent.append(new_data, ignore_index=True)

In [19]:
senti_aspect_percent

,product_name,sooth,moisture,cost,func
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),44.74,32.89,22.37,0.0
1,에스트라 아토베리어365 크림 80ml 짝꿍기획 (+하이드로에센스 40ml 증정),31.58,68.42,0.00,0.0
2,[단독][1+1] 크리니크 쏙보습크림 50ml 1+1 기획세트 (+애슝 스티커 증정),0.00,100.00,0.00,0.0
3,"라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)",80.49,2.44,17.07,0.0
4,[오원Pick] 닥터자르트 시카페어 크림(50ml+25ml) 한정세트,39.13,17.39,43.48,0.0
...,...,...,...,...,...
218,메이크프렘 세이프 미 릴리프 인텐시브 CP 크림 80ml,30.77,46.15,23.08,0.0
219,피지오겔 레드수딩 AI 인텐시브 리피드크림 50ml,66.67,0.00,33.33,0.0
220,웰라쥬 리얼 히알루로닉 겔크림 75ml,0.00,100.00,0.00,0.0
221,닥터디퍼런트 131 모이스처라이저 100ml,0.00,0.00,0.00,100.0


In [21]:
senti_aspect_percent.to_csv('senti_aspect_percent_oily.csv', index = False, columns = ['product_name', 'sooth','moisture','cost','func'])

### 속성별 긍정 비율 (%)

In [22]:
senti_pos_percent = pd.DataFrame(columns=['product_name', 'sooth','moisture','cost','func'])

In [24]:
for i in range(len(senti_product)):
    s_per=0
    m_per=0
    c_per=0
    f_per=0
    
    if sum(senti_product['sooth'][i]) != 0:
        s_per=round(senti_product['sooth'][i][1]/sum(senti_product['sooth'][i]), 4)
    
    if sum(senti_product['moisture'][i]) != 0 :
        m_per=round(senti_product['moisture'][i][1]/sum(senti_product['moisture'][i]),4)
    
    if sum(senti_product['cost'][i]) != 0:
        c_per=round(senti_product['cost'][i][1]/sum(senti_product['cost'][i]),4)
    
    if sum(senti_product['func'][i]) != 0:
        f_per=round(senti_product['func'][i][1]/sum(senti_product['func'][i]),4)
    
    new_data = {
        'product_name': senti_product['product_name'][i],
        'sooth': s_per*100,
        'moisture': m_per*100,
        'cost': c_per*100,
        'func': f_per*100
    }
    
    senti_pos_percent = senti_pos_percent.append(new_data, ignore_index=True)

In [25]:
senti_pos_percent

,product_name,sooth,moisture,cost,func
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),55.88,64.00,47.06,0
1,에스트라 아토베리어365 크림 80ml 짝꿍기획 (+하이드로에센스 40ml 증정),50.00,53.85,0.00,0
2,[단독][1+1] 크리니크 쏙보습크림 50ml 1+1 기획세트 (+애슝 스티커 증정),0.00,100.00,0.00,0
3,"라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)",54.55,100.00,85.71,0
4,[오원Pick] 닥터자르트 시카페어 크림(50ml+25ml) 한정세트,44.44,50.00,80.00,0
...,...,...,...,...,...
218,메이크프렘 세이프 미 릴리프 인텐시브 CP 크림 80ml,25.00,50.00,33.33,0
219,피지오겔 레드수딩 AI 인텐시브 리피드크림 50ml,100.00,0.00,0.00,0
220,웰라쥬 리얼 히알루로닉 겔크림 75ml,0.00,100.00,0.00,0
221,닥터디퍼런트 131 모이스처라이저 100ml,0.00,0.00,0.00,100.0


In [27]:
senti_pos_percent.to_csv('senti_pos_percent_oily.csv', index = False, columns = ['product_name', 'sooth','moisture','cost','func'])